## **Importar librerías**
Importar las librerías necesarias para el análisis de datos y el procesamiento del lenguaje natural,

In [2]:
import numpy as np
import pandas as pd 
import spacy 

## **Cargar los datos**
Cargar los datos desde un archivo CSV llamado "dfListo.csv" y crear una muestra aleatoria de 1000 películas.

In [11]:
# cargar los datos
df = pd.read_csv('dfListo.csv')

# limitar el conjunto de datos de entrenamiento
muestra = df.sample(n=1000, random_state=42)

## **Procesamiento del lenguaje natural**
Cargar el modelo de lenguaje de spaCy y procesar las sinopsis de las películas para obtener una matriz de características de embeddings.

In [12]:
# cargar el modelo de lenguaje de spaCy
nlp = spacy.load("en_core_web_md")

# procesar las sinopsis de las películas
docs = list(nlp.pipe(muestra["overview"].fillna("").values))

# generar una matriz de características de embeddings de spaCy
embedding_matrix = np.array([doc.vector for doc in docs])

## **Cálculo de similitud coseno**
Calcular la matriz de similitud coseno entre las películas basada en su matriz de características de embeddings.

In [13]:
# calcular la matriz de similitud coseno
cosine_sim = np.dot(embedding_matrix, embedding_matrix.T)

## **Eliminar filas duplicadas**
Eliminar las filas duplicadas del conjunto de datos de entrenamiento.

In [14]:
# eliminar las filas duplicadas del conjunto de datos de entrenamiento
muestra = muestra.drop_duplicates().reset_index(drop=True)

## **Búsqueda de películas similares**
Crear una serie de índices para buscar películas por título y definir una función para buscar las películas más similares a una película de entrada.

In [19]:
# crear una serie de índices para buscar películas por título
indices = pd.Series(muestra.index, index=muestra['title'])

# definir una función para buscar películas similares
def recomendacion(title, cosine_sim=cosine_sim, muestra=muestra, indices=indices):
    # encontrar el índice de la película de entrada
    idx = indices[title]

    # calcular las similitudes de la película de entrada con todas las demás películas
    sim_scores = list(enumerate(cosine_sim[idx]))

    # ordenar las películas por similitud descendente
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # seleccionar las cinco películas más similares (excluyendo la película de entrada)
    sim_scores = sim_scores[1:6]

    # obtener los índices de las películas seleccionadas
    movie_indices = [i[0] for i in sim_scores]

    # obtener los títulos de las películas seleccionadas
    peliculas_recomendadas = list(muestra.iloc[movie_indices]['title'])

    # devolver los títulos de las películas recomendadas
    return {'peliculas_recomendadas': peliculas_recomendadas}

## **Ejemplo de uso**
Ejemplo de uso de la función "get_similar_movies" para buscar las películas más similares a "Statues Also Die".

In [20]:
recomendacion('Statues Also Die')

{'peliculas_recomendadas': ['Hated:  GG Allin & the Murder Junkies',
  'This Land Is Mine',
  'The Last Truck: Closing of a GM Plant',
  'Culloden',
  'Richard III']}